## Deployment

Какво е deployment? В изолация, всяко парче код е безполезно, включително и ML/AI. Deployment е процесът по "отваряне" на кода, така че други системи или потребители да могат да достъпват нашия софтуер 

## Трениране на модел
Ще натренираме модел, който използва отворен dataset, за да оцени дали дадено ревю за филм е позитивно или негативно.

In [1]:
from datasets import load_dataset
dataset = "sepidmnorozy/Bulgarian_sentiment"
ds = load_dataset(dataset, split="train")
ds['text'][:5]

['Невероятен!!! Ако искате да гледате нещо нестандартно, не се двоумете - това е филмът за вас ;))',
 'Много добре направен :)',
 'Определено най-добрият Хобит!',
 'Муден старт, но нататък е хубав, ненатоварващ и приятен!',
 'Невероятна поредица.']

Първо, трябва да превърнем текста в ML-readable формат, тоест във вектор

In [2]:
from preprocessing import Tokenizer, TokenizingDataset
vocab_size = 350    # number of tokens
tokenizer = Tokenizer(vocab_size)
tokenizer.fit(ds['text'])
train_ds = TokenizingDataset(ds, tokenizer)
train_ds[0]

(tensor([348,  51, 102, 229,  97,  57, 311, 159,  56, 165, 116, 135, 139, 116,
         107, 184, 107,  67, 221, 286,   2,  35,  70,  71,  95, 136,  17, 214,
           7, 164, 116,  28,   5, 115, 142,  61, 216, 114,  83, 178,  55,  30,
          30, 347, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349,
         349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349,
         349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349],
        dtype=torch.int32),
 tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  

Тренираме модел

In [ ]:
from training_procedure import fit_model
model = fit_model(train_ds)

epoch_loss=903.4081373512745
epoch accuracy: 0.6572431633407243
epoch_loss=881.0009697079659
epoch accuracy: 0.8370288248337029
epoch_loss=902.2710891962051
epoch accuracy: 0.8398004434589801
epoch_loss=902.9718952327967
epoch accuracy: 0.8342572062084257
epoch_loss=930.4109991937876
epoch accuracy: 0.8433111603843311
epoch_loss=939.8380871489644
epoch accuracy: 0.8418329637841833
epoch_loss=917.9347029030323
epoch accuracy: 0.8442350332594235
epoch_loss=921.8313402980566
epoch accuracy: 0.8473762010347377
epoch_loss=926.080776270479
epoch accuracy: 0.8495934959349594
epoch_loss=889.6333800554276
epoch accuracy: 0.8516260162601627
epoch_loss=911.191862154752
epoch accuracy: 0.8562453806356245
epoch_loss=886.0780337192118
epoch accuracy: 0.8590169992609017
epoch_loss=873.2165246680379
epoch accuracy: 0.8608647450110865
epoch_loss=895.6323585156351
epoch accuracy: 0.8680709534368071
epoch_loss=862.9621351677924
epoch accuracy: 0.8669623059866962
epoch_loss=857.3540775310248
epoch accurac

За да не се налага да тренираме всеки път, запазваме нашите тренирани компоненти

In [ ]:
import torch
tokenizer.save("tokenizer.pkl")
torch.save(model, "model.pt")

За да използваме модела:

In [ ]:
model(torch.IntTensor([[10, 15, 20, 25]]))

## Inference

In [ ]:
import torch
class SentimentInference():
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def __call__(self, inp: str):
        tokens = self.tokenizer.tokenize(inp)[:self.model.context_size+1]
        res = self.model(torch.IntTensor([tokens]))
        return res.argmax().item()
    
    @staticmethod
    def load():
        m = torch.load("model.pt")
        t = Tokenizer.load("tokenizer.pkl")
        return SentimentInference(m, t)
inference = SentimentInference.load()
inference("скучен и глупав филм")

## Отваряне на достъпа
Следва да пуснем този модел на прод. Нашето решение ще бъде:
- в облака
- безплатно
- достъпно

In [ ]:
from fastapi import FastAPI
from model import SentimentInference
from pydantic import BaseModel


class Review(BaseModel):
    text: str
    username: str

app = FastAPI()
inf = SentimentInference.load()

@app.post("/sentiment")
async def infer_sentiment(req: Review):
    res = inf(req.text)
    response = "negative" if res == 0 else "positive"
    return {"sentiment": response}

# Какво липсва в тази презентация?
- automated deploy - понякога се налага периодично да тренираме нов модел и той автоматично да се деплойва вместо ръчно да го качваме по всички наши сървъри
- security - всеки може да достъпва нашата виртуална машина. Нямаме никаква защита против атака, примерно DDOS
- scale - моделът е прост и има само една инстанция. Обикновено се налага да имаме възможност да осигурим повече ресурси
- docker и kubernetes - един сървиз би влязъл в контейнер под kubernetes. На практика, няма да се налага да правим ръчните команди по сетване на нова виртуалка, а нашия сървиз би бил по-secure и по-manageable откъм monitoring, logging, automated scaling, etc
- stress test - на какъв брой рекуести може да издържи нашия сървиз
- monitoring - как да разберем възможно най-бързо когато има проблем със сървиза или AWS
- networking - важна тема, но реших да не разводнявам дискусията. Добра идея е поне да фиксираме адреса на нашия сървиз, така че да не го викаме по IP